In [1]:
%%capture

import sys
!{sys.executable} -m pip install quantum-image-classifier

# Quantum image classifier
## Package usage

If you want to use this package, first you will need to download ir from https://github.com/jorgevazquezperez/Quantum-image-classifier. After that, you can import it and start using it as every other package.

In [2]:
import quantum_image_classifier

## Features
Here we will explain the several options that this package offers, in order to generate a template which you could use to try it yourself.

### Data treatment
You can obtain the data you want to test the algorithms in this section of the package. First of all, you can generate synthetic data by using the function `generate_synthetic_data` like we show you here.

In [3]:
from quantum_image_classifier import data_generator as dg

n_dim = 8
n_clusters = 4
n_samples = 250

train_X, train_y, test_X, test_y = dg.generate_synthetic_data(n_dim, n_clusters, n_samples)

Other option is to use a dataset of your own. To do this you need to be aware of a very important aspect of the nature of the quantum algorithms: **you can operate with a very limited number of qubits**. This means that, if you wish to classify an actual image, you need to preprocess it before applying the algorithms in order to reduce its dimensionality and be able to use the data (we recommend this to data with a dimension greater than 32). To do this you can use one of the three options we give you:

1. **Use PCA.** You can use PCA method to achieve a dimension reduction. The only problem this method has is that you have to assume a linear relationship between the input and the output, so, if there is a non-linear relationship, you can generate non-accurate data.
2. **Use autoencoders (AE).** You can use autoencoders in order to achieve a dimension reduction. This solves the non-linear cases, but increases the execution time, because you have to train a neural network.
3. **Input your own preprocess method.** If you don't to use our methods, it's OK (before that I got to ask you: are you sure?). You can insert a custom function which will preprocess the data. ***COMPLETAR***

## Algorithms
### Nearest centroid algorithm

In [4]:
from quantum_image_classifier import NearestCentroid

nearest_centroid = NearestCentroid(train_X, train_y, n_dim)
labels_predicted = nearest_centroid.predict(test_X)

print(test_y)
print(labels_predicted)

[3 1 3 2 2 2 2 1 1 3 2 3 2 1 1 2 1 2 2 3 0 1 1 3 0 1 0 2 1 2 3 0 3 1 1 2 1
 1 0 3 2 1 0 3 0 3 0 3 0 3 0 0 1 2 0 1 2 2 0 1 2 2 0]
[3 1 3 2 2 2 2 1 1 3 2 3 2 1 1 2 1 2 2 3 0 1 1 3 0 1 0 2 1 2 3 0 3 1 1 2 1
 1 0 3 2 1 0 3 0 3 0 3 0 3 0 0 1 2 0 1 2 2 0 1 2 2 0]
